### 회원정보 가져와 저장하기
- 파일: 회원.xlsx

In [55]:
# 필요 모듈 연결
import pandas as pd
import pymysql as mysql
import time
import sys

In [61]:
connect = mysql.connect(host = 'localhost', port = 3306, db = 'test',
                       user = 'root', password = 'Smileboy1!', autocommit = True)

cursor = connect.cursor()

# 저장할 데이터 가져오기
df = pd.read_excel('./회원.xlsx', sheet_name = '기본정보')
df.shape

# sql 구문 작성
cnt = 0
for i in df.index:
    sqlnum = "select num from member where num = '" + df.loc[i, '관리번호'] + "'"
    cursor.execute(sqlnum)
    if len(cursor.fetchall()) == 0:
        sql = "insert into member values('" + df.loc[i, "관리번호"] + "',"    
        sql =  sql + "'" + df.loc[i, "회원구분"] + "',"
        sql =  sql + "'" + df.loc[i, "이름"] + "',"
        sql =  sql + "'" + df.loc[i, "주민등록번호"] + "',"
        sql =  sql + "'" + df.loc[i, "직장명"] + "',"
        sql =  sql + "'" + str(df.loc[i, "생년월일"]) + "',"
        sql =  sql + "'" + df.loc[i, "Tel"] + "',"
        sql =  sql + "'" + df.loc[i, "Hp"] + "',"
        sql =  sql + "'" + str(df.loc[i, "가입일"]) + "')"
        cursor.execute(sql)
        cnt += 1
        
print(f"데이터 {cnt}개 추가")

connect.commit()
connect.close()

In [65]:
str(df.loc[0, '생년월일'])[:10]


'1977-02-02'

In [67]:
# 회원 파일에서 상세정보 시트 내용 가져오기
df = pd.read_excel('./회원.xlsx', sheet_name = '상세정보')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   관리번호    8 non-null      object
 1   결혼여부    8 non-null      object
 2   운전면허증   8 non-null      object
 3   취미      8 non-null      object
 4   특기(소질)  8 non-null      object
 5   관심분야    8 non-null      object
 6   본적      8 non-null      object
 7   주민등록상   8 non-null      object
 8   비고      3 non-null      object
dtypes: object(9)
memory usage: 704.0+ bytes


In [70]:
# 비고 열 NaN 값 확인
df['비고'].isna().sum()

0

In [71]:
# 비고 열 NaN 값을 ""로 변경
df = df.fillna("")
df['비고'].isna().sum()

0

In [110]:
import time
timeS = time.time()

# test DB 연결
connect = mysql.connect(host = 'localhost', port = 3306, db = 'test',
                       user = 'root', password = 'Smileboy1!', autocommit = True)

cursor = connect.cursor()

for i in range(len(df)):
    sql = 'insert into meminfo values(%s, %s, %s, %s, %s, %s)'
    cursor.execute(sql, (df.iloc[i,0], df.iloc[i,1] , df.iloc[i,4],
                         df.iloc[i,5], df.iloc[i,6] , df.iloc[i,8]))
    connect.commit()
print('한개씩 저장 완료', time.time() - timeS)
print('DB 저장완료')

connect.commit()
connect.close()

한개씩 저장 완료 0.0499119758605957
DB 저장완료


In [111]:
# 튜플구조의 데이터를 생성해서 한번에 여러개의 데이터를 추가하는 방법
timeS = time.time()
connect = mysql.connect(host = 'localhost', port = 3306, db = 'test',
                       user = 'root', password = 'Smileboy1!', autocommit = True)

cursor = connect.cursor()
sql = 'delete from meminfo'

cursor.execute(sql)
connect.commit()
connect.close()

In [112]:
# 튜플구조의 데이터를 생성해서 한번에 여러개의 데이터를 추가하는 방법
timeS = time.time()
connect = mysql.connect(host = 'localhost', port = 3306, db = 'test',
                       user = 'root', password = 'Smileboy1!', autocommit = True)
cursor = connect.cursor()

sql_lst = []
for i in range(len(df)):
    sql_lst.append((df.iloc[i,0], df.iloc[i,1] , df.iloc[i,4],
                         df.iloc[i,5], df.iloc[i,6] , df.iloc[i,8]))
    
sql = 'insert into meminfo values(%s, %s, %s, %s, %s, %s)'
cursor.executemany(sql, tuple(sql_lst))

print('한번에 저장 완료', time.time() - timeS)
print('DB저장완료')
connect.commit()
connect.close()

한번에 저장 완료 0.04459500312805176
DB저장완료
